# Pymesh --> Skeleton

In [1]:
#import modules and set up datajoint

In [2]:
import datajoint as dj
import numpy as np
import calcification_Module as cm
import os
import pathlib
import subprocess
import time

#for the shrinkwrapping:
import pymeshfix

#for supressing the output
import os, contextlib


In [3]:
#connect to the database and pull down Cleansed Mesh ones and generate the resurfacing of these
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [4]:
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/temp")) == False:
    os.mkdir("temp")

In [5]:
def skeletonize_Neuron_Meshlabserver(key):
    start_time_inner = time.time()
    vertices_ar,triangles_ar = (ta3p100.Decimation35 & key).fetch("vertices","triangles")

    vertices = vertices_ar[0]
    triangles = triangles_ar[0]
    print(f"Import time = {time.time() - start_time_inner}")
 


    start_time_inner = time.time()
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    #run the mesh through Pymesh to make manifold and 
    meshfix = pymeshfix.MeshFix(vertices,triangles)
    meshfix.repair(verbose=False,joincomp=False,remove_smallest_components=False)
    #vertices_new = meshfix.v
    #faces_new = meshfix.f
    #write_Whole_Neuron_Off_file(str(segment_id) + pymesh_type,vertices_new,faces_new)
    print(f"Pymeshfix shrinkwrap took {time.time()-start_time_inner} seconds")
    
    
    start_time_inner = time.time()
    #try the inline printing method:
    with open(os.devnull, 'w') as devnull:
        with contextlib.redirect_stdout(devnull):
            meshfix.export(str(path_and_filename) + ".off")
    print(f"Writing Off file = {time.time() - start_time_inner}")
    
    
    #save off the voxelized mesh as .off 
    #so it can be passed to meshlab server
    
    """  SLOWER THAN THE EXPORT FUNCTION OF THE VOXEL_MESH BY 2 SECONDS
    start_time_inner = time.time()
    print("starting writing neuron to off file")
    paths = write_Whole_Neuron_Off_file(str(key['segment_id'])+"_vox",voxel_mesh.vertices,voxel_mesh.faces)
    print("DONE writing neuron to off file")
    print(f"Writing Off file time = {time.time() - start_time_inner}")
    """
    
    """
    start_time_inner = time.time()
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    meshlab_script = str(file_loc) + "/" + "remeshing_script_vox.mlx"
    #send to meshlabserver
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    print("subprocess_result= "+ str(subprocess_result))
    print("DONE meshlabserver Poisson surface reconstruction")
    print(f"Meshlabserver time = {time.time() - start_time_inner}")
    """
    
    #skeletonize the mesh
    start_time_inner = time.time()
    print("starting creating skeleton")
    calc_return_value = cm.calcification(path_and_filename)
    print("calc_return_value = " + str(calc_return_value))
    print("DONE creating skeleton")
    print(f"Skeleton time = {time.time() - start_time_inner}")
    
    return calc_return_value

    

In [6]:

segment_id = 648518346341366885
label_key = dict(segment_id=segment_id)
start_time = time.time()

skeletonize_Neuron_Meshlabserver(label_key)

print(f"Total time for Meshlabserver = {time.time() - start_time}")


Import time = 2.5767433643341064
Pymeshfix shrinkwrap took 58.28129529953003 seconds


AttributeError: 'MeshFix' object has no attribute 'export'